In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# read enrollment data per year, then concatenate
df_20 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2020.csv', sep="|")
df_21 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2021.csv', sep="|")
df_22 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2022.csv', sep="|")
df_23 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2023.csv', sep="|")
df_24 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2024.csv', sep="|")
df_25 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2025.csv', sep="|")

enrollment = pd.concat([df_20,df_21,df_22,df_23,df_24,df_25], ignore_index=True)
enrollment.head()

<ipython-input-2-3018ea6ded42>:8: DtypeWarning: Columns (161,162,163,164,165,166,167,168,169,170,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  df_21 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2021.csv', sep="|")
<ipython-input-2-3018ea6ded42>:10: DtypeWarning: Columns (161,162,163,164,165,166,167,168,169,170,171,172) have mixed types. Specify dtype option on import or set low_memory=False.
  df_23 = pd.read_csv('/content/drive/MyDrive/Data/Input/All Beneficiary Years/beneficiary_2023.csv', sep="|")


,BENE_ID,STATE_CODE,COUNTY_CD,ZIP_CD,BENE_BIRTH_DT,SEX_IDENT_CD,BENE_RACE_CD,ENTLMT_RSN_ORIG,ENTLMT_RSN_CURR,ESRD_IND,BENE_PTA_TRMNTN_CD,BENE_PTB_TRMNTN_CD,BENE_DEATH_DT,BENE_ENROLLMT_REF_YR,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_STATE_BUYIN_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,RDS_CVRG_MONS,ENRL_SRC,SAMPLE_GROUP,ENHANCED_FIVE_PERCENT_FLAG,CRNT_BIC_CD,AGE_AT_END_REF_YR,COVSTART,DUAL_ELGBL_MONS,STATE_CNTY_FIPS_CD_01,STATE_CNTY_FIPS_CD_02,STATE_CNTY_FIPS_CD_03,STATE_CNTY_FIPS_CD_04,STATE_CNTY_FIPS_CD_05,STATE_CNTY_FIPS_CD_06,STATE_CNTY_FIPS_CD_07,STATE_CNTY_FIPS_CD_08,STATE_CNTY_FIPS_CD_09,STATE_CNTY_FIPS_CD_10,STATE_CNTY_FIPS_CD_11,STATE_CNTY_FIPS_CD_12,VALID_DEATH_DT_SW,RTI_RACE_CD,MDCR_STATUS_CODE_01,MDCR_STATUS_CODE_02,MDCR_STATUS_CODE_03,MDCR_STATUS_CODE_04,MDCR_STATUS_CODE_05,MDCR_STATUS_CODE_06,MDCR_STATUS_CODE_07,MDCR_STATUS_CODE_08,MDCR_STATUS_CODE_09,MDCR_STATUS_CODE_10,MDCR_STATUS_CODE_11,MDCR_STATUS_CODE_12,PTD_PLAN_CVRG_MONS,MDCR_ENTLMT_BUYIN_IND_01,MDCR_ENTLMT_BUYIN_IND_02,MDCR_ENTLMT_BUYIN_IND_03,MDCR_ENTLMT_BUYIN_IND_04,MDCR_ENTLMT_BUYIN_IND_05,MDCR_ENTLMT_BUYIN_IND_06,MDCR_ENTLMT_BUYIN_IND_07,MDCR_ENTLMT_BUYIN_IND_08,MDCR_ENTLMT_BUYIN_IND_09,MDCR_ENTLMT_BUYIN_IND_10,MDCR_ENTLMT_BUYIN_IND_11,MDCR_ENTLMT_BUYIN_IND_12,HMO_IND_01,HMO_IND_02,HMO_IND_03,HMO_IND_04,HMO_IND_05,HMO_IND_06,HMO_IND_07,HMO_IND_08,HMO_IND_09,HMO_IND_10,HMO_IND_11,HMO_IND_12,PTC_CNTRCT_ID_01,PTC_CNTRCT_ID_02,PTC_CNTRCT_ID_03,PTC_CNTRCT_ID_04,PTC_CNTRCT_ID_05,PTC_CNTRCT_ID_06,PTC_CNTRCT_ID_07,PTC_CNTRCT_ID_08,PTC_CNTRCT_ID_09,PTC_CNTRCT_ID_10,PTC_CNTRCT_ID_11,PTC_CNTRCT_ID_12,PTC_PBP_ID_01,PTC_PBP_ID_02,PTC_PBP_ID_03,PTC_PBP_ID_04,PTC_PBP_ID_05,PTC_PBP_ID_06,PTC_PBP_ID_07,PTC_PBP_ID_08,PTC_PBP_ID_09,PTC_PBP_ID_10,PTC_PBP_ID_11,PTC_PBP_ID_12,PTC_PLAN_TYPE_CD_01,PTC_PLAN_TYPE_CD_02,PTC_PLAN_TYPE_CD_03,PTC_PLAN_TYPE_CD_04,PTC_PLAN_TYPE_CD_05,PTC_PLAN_TYPE_CD_06,PTC_PLAN_TYPE_CD_07,PTC_PLAN_TYPE_CD_08,PTC_PLAN_TYPE_CD_09,PTC_PLAN_TYPE_CD_10,PTC_PLAN_TYPE_CD_11,PTC_PLAN_TYPE_CD_12,PTD_CNTRCT_ID_01,PTD_CNTRCT_ID_02,PTD_CNTRCT_ID_03,PTD_CNTRCT_ID_04,PTD_CNTRCT_ID_05,PTD_CNTRCT_ID_06,PTD_CNTRCT_ID_07,PTD_CNTRCT_ID_08,PTD_CNTRCT_ID_09,PTD_CNTRCT_ID_10,PTD_CNTRCT_ID_11,PTD_CNTRCT_ID_12,PTD_PBP_ID_01,PTD_PBP_ID_02,PTD_PBP_ID_03,PTD_PBP_ID_04,PTD_PBP_ID_05,PTD_PBP_ID_06,PTD_PBP_ID_07,PTD_PBP_ID_08,PTD_PBP_ID_09,PTD_PBP_ID_10,PTD_PBP_ID_11,PTD_PBP_ID_12,PTD_SGMT_ID_01,PTD_SGMT_ID_02,PTD_SGMT_ID_03,PTD_SGMT_ID_04,PTD_SGMT_ID_05,PTD_SGMT_ID_06,PTD_SGMT_ID_07,PTD_SGMT_ID_08,PTD_SGMT_ID_09,PTD_SGMT_ID_10,PTD_SGMT_ID_11,PTD_SGMT_ID_12,RDS_IND_01,RDS_IND_02,RDS_IND_03,RDS_IND_04,RDS_IND_05,RDS_IND_06,RDS_IND_07,RDS_IND_08,RDS_IND_09,RDS_IND_10,RDS_IND_11,RDS_IND_12,DUAL_STUS_CD_01,DUAL_STUS_CD_02,DUAL_STUS_CD_03,DUAL_STUS_CD_04,DUAL_STUS_CD_05,DUAL_STUS_CD_06,DUAL_STUS_CD_07,DUAL_STUS_CD_08,DUAL_STUS_CD_09,DUAL_STUS_CD_10,DUAL_STUS_CD_11,DUAL_STUS_CD_12,CST_SHR_GRP_CD_01,CST_SHR_GRP_CD_02,CST_SHR_GRP_CD_03,CST_SHR_GRP_CD_04,CST_SHR_GRP_CD_05,CST_SHR_GRP_CD_06,CST_SHR_GRP_CD_07,CST_SHR_GRP_CD_08,CST_SHR_GRP_CD_09,CST_SHR_GRP_CD_10,CST_SHR_GRP_CD_11,CST_SHR_GRP_CD_12
0,-10000010254618,1,1500,36109,16-Aug-1999,1,1,1,1,0,0,0,NaN,2020,12,12,12,0,0,CME,NaN,,M,21,08-Sep-1999,0,1101.0,1101.0,1101.0,1101.0,1101.0,1101.0,1101.0,1101.0,1101.0,1101.0,1101.0,1101.0,NaN,1,20,20,20,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,12,3,3,3,3,3,3,3,3,3,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y0004,Y0004,Y0004,Y0004,Y0004,Y0004,Y0004,Y0004,Y0004,Y0004,Y0004,Y0004,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z0004,Z0004,Z0004,Z0004,Z0004,Z0004,Z0004,Z0004,Z0004,Z0004,Z0004,Z0004,804.0,804.0,804.0,804.0,804.0,804.0,804.0,804.0,804.0,804.0,804.0,804.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,-10000010254647,1,1410,35756,14-Aug-1990,1,2,1,1,0,0,0,NaN,2020,12,12,12,0,0

In [3]:
# prompt: are there any BENE_ID for whom the 12 columns denoted by MDCR_STATUS_CODE_01 through MDCR_STATUS_CODE_12 equals 0
## See pg. 34 of membership codebook: https://www2.ccwdata.org/documents/10280/19022436/codebook-mbsf-abcd.pdf
mdc_status_cols = [f'MDCR_STATUS_CODE_{i:02d}' for i in range(1, 13)]

beneficiary_with_zero_mdc = enrollment[enrollment[mdc_status_cols].isin([0]).any(axis=1)]['BENE_ID'].unique()
print(beneficiary_with_zero_mdc)

[]


There were no members with gap enrollment

In [4]:
# prompt: for each BENE_ID, is the average value across 12 columns denoted by MDCR_STATUS_CODE_01 through MDCR_STATUS_CODE_12 the same as the value in MDCR_STATUS_CODE_01? Are there exceptions?

# Calculate the average of MDCR_STATUS_CODE columns for each BENE_ID
enrollment['avg_mdc_status'] = enrollment[mdc_status_cols].mean(axis=1)

# Compare the average with MDCR_STATUS_CODE_01
enrollment['avg_equals_first'] = enrollment['avg_mdc_status'] == enrollment['MDCR_STATUS_CODE_01']

# Identify exceptions
exceptions = enrollment[~enrollment['avg_equals_first']]
exceptions

,BENE_ID,STATE_CODE,COUNTY_CD,ZIP_CD,BENE_BIRTH_DT,SEX_IDENT_CD,BENE_RACE_CD,ENTLMT_RSN_ORIG,ENTLMT_RSN_CURR,ESRD_IND,BENE_PTA_TRMNTN_CD,BENE_PTB_TRMNTN_CD,BENE_DEATH_DT,BENE_ENROLLMT_REF_YR,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_STATE_BUYIN_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,RDS_CVRG_MONS,ENRL_SRC,SAMPLE_GROUP,ENHANCED_FIVE_PERCENT_FLAG,CRNT_BIC_CD,AGE_AT_END_REF_YR,COVSTART,DUAL_ELGBL_MONS,STATE_CNTY_FIPS_CD_01,STATE_CNTY_FIPS_CD_02,STATE_CNTY_FIPS_CD_03,STATE_CNTY_FIPS_CD_04,STATE_CNTY_FIPS_CD_05,STATE_CNTY_FIPS_CD_06,STATE_CNTY_FIPS_CD_07,STATE_CNTY_FIPS_CD_08,STATE_CNTY_FIPS_CD_09,STATE_CNTY_FIPS_CD_10,STATE_CNTY_FIPS_CD_11,STATE_CNTY_FIPS_CD_12,VALID_DEATH_DT_SW,RTI_RACE_CD,MDCR_STATUS_CODE_01,MDCR_STATUS_CODE_02,MDCR_STATUS_CODE_03,MDCR_STATUS_CODE_04,MDCR_STATUS_CODE_05,MDCR_STATUS_CODE_06,MDCR_STATUS_CODE_07,MDCR_STATUS_CODE_08,MDCR_STATUS_CODE_09,MDCR_STATUS_CODE_10,MDCR_STATUS_CODE_11,MDCR_STATUS_CODE_12,PTD_PLAN_CVRG_MONS,MDCR_ENTLMT_BUYIN_IND_01,MDCR_ENTLMT_BUYIN_IND_02,MDCR_ENTLMT_BUYIN_IND_03,MDCR_ENTLMT_BUYIN_IND_04,MDCR_ENTLMT_BUYIN_IND_05,MDCR_ENTLMT_BUYIN_IND_06,MDCR_ENTLMT_BUYIN_IND_07,MDCR_ENTLMT_BUYIN_IND_08,MDCR_ENTLMT_BUYIN_IND_09,MDCR_ENTLMT_BUYIN_IND_10,MDCR_ENTLMT_BUYIN_IND_11,MDCR_ENTLMT_BUYIN_IND_12,HMO_IND_01,HMO_IND_02,HMO_IND_03,HMO_IND_04,HMO_IND_05,HMO_IND_06,HMO_IND_07,HMO_IND_08,HMO_IND_09,HMO_IND_10,HMO_IND_11,HMO_IND_12,PTC_CNTRCT_ID_01,PTC_CNTRCT_ID_02,PTC_CNTRCT_ID_03,PTC_CNTRCT_ID_04,PTC_CNTRCT_ID_05,PTC_CNTRCT_ID_06,PTC_CNTRCT_ID_07,PTC_CNTRCT_ID_08,PTC_CNTRCT_ID_09,PTC_CNTRCT_ID_10,PTC_CNTRCT_ID_11,PTC_CNTRCT_ID_12,PTC_PBP_ID_01,PTC_PBP_ID_02,PTC_PBP_ID_03,PTC_PBP_ID_04,PTC_PBP_ID_05,PTC_PBP_ID_06,PTC_PBP_ID_07,PTC_PBP_ID_08,PTC_PBP_ID_09,PTC_PBP_ID_10,PTC_PBP_ID_11,PTC_PBP_ID_12,PTC_PLAN_TYPE_CD_01,PTC_PLAN_TYPE_CD_02,PTC_PLAN_TYPE_CD_03,PTC_PLAN_TYPE_CD_04,PTC_PLAN_TYPE_CD_05,PTC_PLAN_TYPE_CD_06,PTC_PLAN_TYPE_CD_07,PTC_PLAN_TYPE_CD_08,PTC_PLAN_TYPE_CD_09,PTC_PLAN_TYPE_CD_10,PTC_PLAN_TYPE_CD_11,PTC_PLAN_TYPE_CD_12,PTD_CNTRCT_ID_01,PTD_CNTRCT_ID_02,PTD_CNTRCT_ID_03,PTD_CNTRCT_ID_04,PTD_CNTRCT_ID_05,PTD_CNTRCT_ID_06,PTD_CNTRCT_ID_07,PTD_CNTRCT_ID_08,PTD_CNTRCT_ID_09,PTD_CNTRCT_ID_10,PTD_CNTRCT_ID_11,PTD_CNTRCT_ID_12,PTD_PBP_ID_01,PTD_PBP_ID_02,PTD_PBP_ID_03,PTD_PBP_ID_04,PTD_PBP_ID_05,PTD_PBP_ID_06,PTD_PBP_ID_07,PTD_PBP_ID_08,PTD_PBP_ID_09,PTD_PBP_ID_10,PTD_PBP_ID_11,PTD_PBP_ID_12,PTD_SGMT_ID_01,PTD_SGMT_ID_02,PTD_SGMT_ID_03,PTD_SGMT_ID_04,PTD_SGMT_ID_05,PTD_SGMT_ID_06,PTD_SGMT_ID_07,PTD_SGMT_ID_08,PTD_SGMT_ID_09,PTD_SGMT_ID_10,PTD_SGMT_ID_11,PTD_SGMT_ID_12,RDS_IND_01,RDS_IND_02,RDS_IND_03,RDS_IND_04,RDS_IND_05,RDS_IND_06,RDS_IND_07,RDS_IND_08,RDS_IND_09,RDS_IND_10,RDS_IND_11,RDS_IND_12,DUAL_STUS_CD_01,DUAL_STUS_CD_02,DUAL_STUS_CD_03,DUAL_STUS_CD_04,DUAL_STUS_CD_05,DUAL_STUS_CD_06,DUAL_STUS_CD_07,DUAL_STUS_CD_08,DUAL_STUS_CD_09,DUAL_STUS_CD_10,DUAL_STUS_CD_11,DUAL_STUS_CD_12,CST_SHR_GRP_CD_01,CST_SHR_GRP_CD_02,CST_SHR_GRP_CD_03,CST_SHR_GRP_CD_04,CST_SHR_GRP_CD_05,CST_SHR_GRP_CD_06,CST_SHR_GRP_CD_07,CST_SHR_GRP_CD_08,CST_SHR_GRP_CD_09,CST_SHR_GRP_CD_10,CST_SHR_GRP_CD_11,CST_SHR_GRP_CD_12,avg_mdc_status,avg_equals_first
81,-10000010254979,1,1080,36863,17-May-1955,1,2,1,0,0,0,0,NaN,2020,12,12,12,0,0,CME,NaN,,A,65,08-May-1989,0,1017.0,1017.0,1017.0,1017.0,1017.0,1017.0,1017.0,1017.0,1017.0,1017.0,1017.0,1017.0,NaN,2,20,20,20,20.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,12,3,3,3,3,3,3,3,3,3,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y0007,Y0007,Y0007,Y0007,Y0007,Y0007,Y0007,Y0007,Y0007,Y0007,Y0007,Y0007,803.0,803.0,803.0,803.0,803.0,803.0,803.0,803.0,803.0,803.0,803.0,803.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,Z0009,Z0009,Z0009,Z0009,Z0009,Z0009,Z0009,Z0009,NaN,NaN,NaN,NaN,801.0,801.0,801.0,801.0,801.0,801.0,801.0,801.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,N,N,N,N,N,N,N,N,00,00,00,00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,13.333333,False
175,-10000010255390,3,3060,85004,02-Oct-1955,1,1,1,0,0,0,

In [5]:
# filter for columns needed
columns_to_keep = ['BENE_ID', 'STATE_CODE', 'COUNTY_CD', 'ZIP_CD', 'BENE_BIRTH_DT', 'SEX_IDENT_CD', 'BENE_RACE_CD', 'ESRD_IND', 'BENE_DEATH_DT', 'BENE_ENROLLMT_REF_YR']
enrollment = enrollment[columns_to_keep]
enrollment

,BENE_ID,STATE_CODE,COUNTY_CD,ZIP_CD,BENE_BIRTH_DT,SEX_IDENT_CD,BENE_RACE_CD,ESRD_IND,BENE_DEATH_DT,BENE_ENROLLMT_REF_YR
0,-10000010254618,1,1500,36109,16-Aug-1999,1,1,0,NaN,2020
1,-10000010254647,1,1410,35756,14-Aug-1990,1,2,0,NaN,2020
2,-10000010254653,1,1400,36801,18-Mar-1982,2,1,0,NaN,2020
3,-10000010254656,1,1360,35216,24-Jul-1999,2,1,0,NaN,2020
4,-10000010254666,1,1440,35805,23-Feb-2009,2,4,0,NaN,2020
...,...,...,...,...,...,...,...,...,...,...
53588,-10000010288006,5,5330,95459,14-Sep-1942,1,1,0,NaN,2025
53589,-10000010288007,5,5060,94598,11-Sep-1962,2,1,Y,NaN,2025
53590,-10000010288008,5,5430,92282,26-Dec-1945,1,5,0,NaN,2025
53591,-10000010288009,5,5600,95380,24-Sep-1957,1,1,0,NaN,2025


In [6]:
# prompt: what is the count of BENE_ID by BENE_ENROLLMT_REF_YR
enrollment_count = enrollment.groupby('BENE_ENROLLMT_REF_YR')['BENE_ID'].count()
enrollment_count

,BENE_ID
BENE_ENROLLMT_REF_YR,
2020,7837
2021,8246
2022,8671
2023,9179
2024,9660
2025,10000


In [7]:
# prompt: are there duplicates per BENE_ID and BENE_ENROLLMT_REF_YR?

# Check for duplicates based on 'BENE_ID' and 'BENE_ENROLLMT_REF_YR'
duplicates = enrollment[enrollment.duplicated(subset=['BENE_ID', 'BENE_ENROLLMT_REF_YR'], keep=False)]

if not duplicates.empty:
  print("Duplicate entries found:")
  print(duplicates[['BENE_ID', 'BENE_ENROLLMT_REF_YR']])
else:
  print("No duplicate entries found based on BENE_ID and BENE_ENROLLMT_REF_YR.")


No duplicate entries found based on BENE_ID and BENE_ENROLLMT_REF_YR.


In [8]:
# prompt: what is the maximum number of records per BENE_ID ?
max_records = enrollment.groupby('BENE_ID')['BENE_ID'].count().max()
print(f"The maximum number of records per BENE_ID is: {max_records}")

The maximum number of records per BENE_ID is: 6


In [9]:
# prompt: of the BENE_IDs with 2025 in BENE_ENROLLMT_REF_YR, how many are also in preceding years?
# Create a set of BENE_IDs from 2025
bene_ids_2025 = set(enrollment[enrollment['BENE_ENROLLMT_REF_YR'] == 2025]['BENE_ID'])

# Count the number of BENE_IDs from 2025 that are also present in other years
years_to_check = [2020,2021,2022,2023,2024]

for year in years_to_check:
  bene_ids_year = set(enrollment[enrollment['BENE_ENROLLMT_REF_YR'] == year]['BENE_ID'])
  common_bene_ids = bene_ids_2025.intersection(bene_ids_year)
  print(f"Number of BENE_IDs from 2025 also in {year}: {len(common_bene_ids)}")

Number of BENE_IDs from 2025 also in 2020: 7837
Number of BENE_IDs from 2025 also in 2021: 8246
Number of BENE_IDs from 2025 also in 2022: 8671
Number of BENE_IDs from 2025 also in 2023: 9179
Number of BENE_IDs from 2025 also in 2024: 9660


In [10]:
# prompt: Read BENE_BIRTH_DT as datetime. Create AGE variable as the difference between year extracted from BENE_BIRTH_DT and BENE_ENROLLMT_REF_YR

# Convert 'BENE_BIRTH_DT' to datetime objects
enrollment['BENE_BIRTH_DT'] = pd.to_datetime(enrollment['BENE_BIRTH_DT'], format='%d-%b-%Y', errors='coerce')
enrollment['BENE_BIRTH_YR'] = enrollment['BENE_BIRTH_DT'].dt.year
enrollment['AGE_CALC'] = enrollment['BENE_ENROLLMT_REF_YR'] - enrollment['BENE_BIRTH_YR']

In [11]:
# prompt: create AGE_GRP from AGE_CALC, AGE_CALC<=18 (AGE_GRP='CHILD'). Else if AGE_CALC<65 ('ADULT BELOW 65'). Else if AGE_CALC>65 ('ADULT 65+')

# Create AGE_GRP based on AGE_CALC
enrollment['AGE_GRP'] = np.where(enrollment['AGE_CALC'] <= 18, 'CHILD', np.where(enrollment['AGE_CALC'] < 65, 'ADULT BELOW 65', 'ADULT 65+'))

# Count the frequency of each AGE_GRP
age_grp_counts = enrollment['AGE_GRP'].value_counts()
age_grp_counts

,count
AGE_GRP,
ADULT 65+,30297
ADULT BELOW 65,20475
CHILD,2821


In [12]:
enrollment = enrollment.drop(columns=['BENE_BIRTH_DT', 'BENE_BIRTH_YR', 'BENE_DEATH_DT'])
enrollment.head()

,BENE_ID,STATE_CODE,COUNTY_CD,ZIP_CD,SEX_IDENT_CD,BENE_RACE_CD,ESRD_IND,BENE_ENROLLMT_REF_YR,AGE_CALC,AGE_GRP
0,-10000010254618,1,1500,36109,1,1,0,2020,21,ADULT BELOW 65
1,-10000010254647,1,1410,35756,1,2,0,2020,30,ADULT BELOW 65
2,-10000010254653,1,1400,36801,2,1,0,2020,38,ADULT BELOW 65
3,-10000010254656,1,1360,35216,2,1,0,2020,21,ADULT BELOW 65
4,-10000010254666,1,1440,35805,2,4,0,2020,11,CHILD


In [13]:
enrollment.to_csv('/content/drive/MyDrive/Data/Output/MemberYear.csv', index=False)